Here we test if we can interate $r_{d}$ for the GILA model.

In [11]:
#Import libraries related to BAO data.
import numpy as np
from numba import jit
from scipy.interpolate import interp1d
from scipy.integrate import cumtrapz as cumtrapz
from scipy.integrate import simps as simps
from scipy.integrate import quad as quad

from scipy.constants import c as c_luz #meters/seconds
c_luz_km = c_luz/1000

import os
import git
path_git = git.Repo('.', search_parent_directories=True).working_tree_dir
path_datos_global = os.path.dirname(path_git)
os.chdir(path_git)
os.sys.path.append('./fr_mcmc/utils/')
from change_of_parameters import omega_CDM_to_luisa, omega_luisa_to_CDM
from LambdaCDM import H_LCDM_rad
from solve_sys import Hubble_th, integrator


Lets define some functions for the calculations of the rd for LCDM

In [ ]:
#Parameters order: omega_m,b,H_0,n

def zdrag(omega_m,H_0,wb=0.0225):
    '''
    wb = 0.0222383 #Planck
    wb = 0.0225 #BBN
    '''
    h = H_0/100
    b1 = 0.313*(omega_m*h**2)**(-0.419)*(1+0.607*(omega_m*h**2)**(0.6748))
    b2 = 0.238*(omega_m*h**2)**0.223
    zd = (1291*(omega_m*h**2)**0.251) * (1+b1*wb**b2) /(1+0.659*(omega_m*h**2)**0.828)
    #zd =1060.31
    return zd

@jit
def integrand(z, Om_m_0, H_0, wb):
    R_bar = wb * 10**5 / 2.473

    Om_r = 4.18343*10**(-5) / (H_0/100)**2
    Om_Lambda = 1 - Om_m_0 - Om_r
    H = H_0 * ((Om_r * (1 + z)**4 + Om_m_0 * (1 + z)**3 + Om_Lambda) ** (1/2))
    return c_luz_km/(H * (3*(1 + R_bar*(1+z)**(-1)))**(1/2))


def r_drag_lcdm(omega_m,H_0,wb = 0.0225, int_z=True): #wb of BBN as default.
    #rd calculation:
    h = H_0/100
    zd = zdrag(omega_m,H_0)
    #R_bar = 31500 * wb * (2.726/2.7)**(-4)
    R_bar = wb * 10**5 / 2.473

    #zd calculation:
    zd = zdrag(omega_m, H_0)
    # zd = 1000
    R_bar = wb * 10**5 / 2.473

    rd_log, _ = quad(lambda z: integrand(z, omega_m, H_0, wb), zd, np.inf)

    return rd_log


Now we check that this work:

In [ ]:
H_0 = 70
omega_m = 0.3

rd_lcdm = r_drag_lcdm(omega_m,H_0,wb = 0.0225)
print(rd_lcdm)

Now we define extra functions in order to calculate rd from GILA model

In [23]:
L_bar = 0.75
b = 2
omega_m_luisa = omega_CDM_to_luisa(b, L_bar, H_0, omega_m)
 

physical_params = [L_bar,b,H_0,omega_m_luisa]
#zs_model, Hs_model = Hubble_th(physical_params, model='GILA',
#                            z_min= zdrag(omega_m,H_0), z_max=np.inf)
zs_model, Hs_model = integrator(physical_params, num_z_points=int(10**5), initial_z = zdrag(omega_m,H_0), final_z=1500)#np.inf)
print(zs_model, Hs_model)

@jit
def integrand_GILA(z, Om_m_0, H_0, wb):
    R_bar = wb * 10**5 / 2.473

    #Om_r = 4.18343*10**(-5) / (H_0/100)**2
    #Om_Lambda = 1 - Om_m_0 - Om_r
    aux = interp1d(zs_model, Hs_model)
    H = aux(z)
    #H = H_0 * ((Om_r * (1 + z)**4 + Om_m_0 * (1 + z)**3 + Om_Lambda) ** (1/2))
    return c_luz_km/(H * (3*(1 + R_bar*(1+z)**(-1)))**(1/2))


def r_drag_GILA(omega_m,H_0,wb = 0.0225, int_z=True): #wb of BBN as default.
    #rd calculation:
    #h = H_0/100
    #zd = zdrag(omega_m,H_0)
    #R_bar = 31500 * wb * (2.726/2.7)**(-4)
    #R_bar = wb * 10**5 / 2.473

    #zd calculation:
    zd = zdrag(omega_m, H_0)
    # zd = 1000
    #R_bar = wb * 10**5 / 2.473

    rd_log, _ = quad(lambda z: integrand_GILA(z, omega_m, H_0, wb), zd, np.inf)

    return rd_log

print(r_drag_GILA(omega_m,H_0,wb = 0.0225, int_z=True))

[1021.25698765] [70.]


/tmp/ipykernel_28001/2626422583.py:12: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "integrand_GILA" failed type inference due to: Untyped global name 'interp1d': Cannot determine Numba type of <class 'type'>

File "../../../../../tmp/ipykernel_28001/2626422583.py", line 18:
<source missing, REPL/exec in use?>

  @jit
/home/matias/.local/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "integrand_GILA" was compiled in object mode without forceobj=True.

File "../../../../../tmp/ipykernel_28001/2626422583.py", line 12:
<source missing, REPL/exec in use?>

  state.func_ir.loc))
/home/matias/.local/lib/python3.7/site-packages/numba/core/object_mode_passes.py:162: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/

ValueError: x and y arrays must have at least 2 entries

Finally, lets compare results

In [ ]:
import os
import git
path_git = git.Repo('.', search_parent_directories=True).working_tree_dir

os.chdir(path_git); os.sys.path.append('./fr_mcmc/utils/')
from change_of_parameters import omega_CDM_to_luisa, omega_luisa_to_CDM
from solve_sys import Hubble_th

L_bar = 0.75
b = 2
H_0 = 70

#omega_m_luisa = 0.9999 + 10**(-5) * omega_m_luisa
#omega_m = omega_luisa_to_CDM(b,L_bar,H_0,omega_m_luisa)
omega_m = 0.3
omega_m_luisa = omega_CDM_to_luisa(b, L_bar, H_0, omega_m)

rd = r_drag_lcdm(omega_m,H_0,wb = 0.0225)

physical_params = [L_bar,b,H_0,omega_m_luisa]
zs_model, Hs_model = Hubble_th(physical_params, model='GILA',
                            z_min= zdrag(omega_m,H_0), z_max=np.inf)
print(rd)

@jit
def integrand_GILA(z, Om_m_0, H_0, wb):
    R_bar = wb * 10**5 / 2.473

    Om_r = 4.18343*10**(-5) / (H_0/100)**2
    Om_Lambda = 1 - Om_m_0 - Om_r
    aux = interp1d(zs_model, Hs_model)
    H = aux(z)
    #H = H_0 * ((Om_r * (1 + z)**4 + Om_m_0 * (1 + z)**3 + Om_Lambda) ** (1/2))
    return c_luz_km/(H * (3*(1 + R_bar*(1+z)**(-1)))**(1/2))


def r_drag_GILA(omega_m,H_0,wb = 0.0225, int_z=True): #wb of BBN as default.
    #rd calculation:
    h = H_0/100
    zd = zdrag(omega_m,H_0)
    #R_bar = 31500 * wb * (2.726/2.7)**(-4)
    R_bar = wb * 10**5 / 2.473

    #zd calculation:
    zd = zdrag(omega_m, H_0)
    # zd = 1000
    R_bar = wb * 10**5 / 2.473

    rd_log, _ = quad(lambda z: integrand_GILA(z, omega_m, H_0, wb), zd, np.inf)

    return rd_log


print(r_drag_GILA(omega_m,H_0,wb = 0.0225, int_z=True))

